In [1]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
from PIL.ImageChops import constant
from Scripts.pywin32_testall import project_root
from scipy.stats import obrientransform
import sys
from matplotlib import pyplot as plt
from pathlib import Path
project_root = Path(__file__).resolve().parents[1] if "__file__" in globals() else Path().resolve().parent
sys.path.append(str(project_root))
from src.utils.eda_utils import missingness,summarize

In [2]:
Root_dir = Path.cwd().parent
data_dir = Path(Root_dir.joinpath('data/raw/tlc/yellow/2023'))
zone_path = Path(Root_dir.joinpath('data/raw/tlc/misc/taxi_zone_lookup.csv'))
file_1 = Path(data_dir.joinpath('yellow_tripdata_2023-01.parquet'))

In [3]:
df = pd.read_parquet(file_1)

In [4]:
numeric_all = df.select_dtypes(include="number").columns.tolist()
exclude_ids = [c for c in ["VendorID","RatecodeID","PULocationID","DOLocationID","payment_type","trip_type"]
               if c in df.columns]
cols = [c for c in numeric_all if c not in exclude_ids]

In [5]:
print(f"Numeric columns selected: {len(cols)} (excluded IDs: {len(exclude_ids)})")

Numeric columns selected: 11 (excluded IDs: 5)


In [6]:
num_sum = summarize(df, cols)

In [7]:
num_sum

,name,count_nonnull,n_missing,%missing,mean,median,std,var,min,q25,q75,max,iqr,skew,kurt,z_outliers
0,airport_fee,2995023,71743,2.3,0.107409,0.00,0.355651,0.126488,-1.25,0.00,0.00,1.25,0.00,2.797109,6.545146,264567
1,congestion_surcharge,2995023,71743,2.3,2.274231,2.50,0.771845,0.595745,-2.50,2.50,2.50,2.50,0.00,-3.508430,12.458352,19718
2,extra,3066766,0,0.0,1.537842,1.00,1.789592,3.202641,-7.50,0.00,2.50,12.50,2.50,1.268648,2.128370,44696
3,fare_amount,3066766,0,0.0,18.367069,12.80,17.807822,317.118522,-900.00,8.60,20.50,1160.10,11.90,3.221210,49.554467,35090
4,improvement_surcharge,3066766,0,0.0,0.982085,1.00,0.183353,0.033618,-1.00,1.00,1.00,1.00,0.00,-10.486738,109.465794,31395
5,mta_tax,3066766,0,0.0,0.488290,0.50,0.103464,0.010705,-0.50,0.50,0.50,53.16,0.00,35.315435,21970.425452,48701
6,passenger_count,2995023,71743,2.3,1.362532,1.00,0.896120,0.803031,0.00,1.00,1.00,9.00,0.00,2.875386,9.514253,70825
7,tip_amount,3066766,0,0.0,3.367941,2.72,3.826759,14.644088,-96.22,1.00,4.20,380.80,3.20,4.223831,92.756546,85668
8,tolls_amount,3066766,0,0.0,0.518491,0.00,2.017579,4.070625,-65.00,0.00,0.00,196.99,0.00,5.389351,78.699510,15729
9,total_amount,3066766,0,0.0,27.020383,20.16,22.163589,491.224675,-751.00,15.40,28.70,1169.40,13.30,2.832829,26.595788,87248


In [8]:
out_path = Root_dir / "reports" / "tables" / "num_summary.csv"
out_path.parent.mkdir(parents=True, exist_ok=True)
num_sum.sort_values("name").to_csv(out_path, index=False)